In [1]:
import pandas as pd

In [27]:
import numpy as np

In [2]:
df = pd.read_csv('sample.csv')

In [3]:
df.head()

,client,time,v1,v2
0,1,1,7,1
1,1,2,3,8
2,1,3,2,7
3,1,4,7,3
4,1,5,7,6


In [4]:
df.set_index('time', inplace=True)

In [5]:
g = df.groupby('client')

In [ ]:
for name,group in g:
    print name
    print group[["v1", "v2"]]
    print '*'*10

In [24]:
g.get_group(1).iloc[:, 1:].as_matrix()

array([[7, 1],
       [3, 8],
       [2, 7],
       [7, 3],
       [7, 6]])

In [16]:
l = df['client'].unique().tolist()
l = [1,2,3,4,5,6,7]

In [17]:
chunks = [l[x:x+3] for x in xrange(0, len(l), 3)]

In [18]:
print chunks

[[1, 2, 3], [4, 5, 6], [7]]


In [32]:
for c in chunks:
    l = []
    max_len = 0
    for ite in c:
        cnt = g.get_group(ite).iloc[:, 1:].as_matrix().shape[0]
        if max_len < cnt:
            max_len = cnt
    for ite in c:
        m = g.get_group(ite).iloc[:, 1:].as_matrix()
        diff = max_len - m.shape[0]
        l.append(np.pad(m, ((0,diff), (0,0)), mode='constant', constant_values=0))
        #l.append(g.get_group(ite).iloc[:, 1:].as_matrix())
    break

In [29]:
print l

[array([[7, 1],
       [3, 8],
       [2, 7],
       [7, 3],
       [7, 6]]), array([[6, 7],
       [8, 5],
       [1, 7],
       [2, 8],
       [9, 1],
       [1, 1]]), array([[7, 1],
       [3, 8],
       [2, 7]])]


In [33]:
batch = np.stack(l, axis=0)

In [36]:
batch.shape

(3, 6, 2)

In [52]:
pl_df = pd.read_csv('../LSTM-autoencoder/22_problem_logs_seq.csv')

In [53]:
pl_df.head()

,id,assignment_id,user_id,assistment_id,problem_id,original,correct,answer_id,answer_text,first_action,...,actions,tutor_strategy_id,tutor_mode,assignment_type,teacher_comment,network_state,path_info,discrete_score,updated_at,sequence_id
0,83251979,390779,131019,97389,186155,1,1.0,NaN,64,0.0,...,--- \n- - start\n - 1317738211496\n- - answer...,136294.0,tutor,ClassAssignment,NaN,NaN,NaN,NaN,NaN,11833
1,83252617,390779,131019,97411,186221,1,1.0,NaN,65,0.0,...,--- \n- - start\n - 1317738254825\n- - answer...,136338.0,tutor,ClassAssignment,NaN,NaN,NaN,NaN,NaN,11833
2,83253196,390779,131019,97495,186473,1,1.0,NaN,452,0.0,...,--- \n- - start\n - 1317738290513\n- - answer...,136506.0,tutor,ClassAssignment,NaN,NaN,NaN,NaN,NaN,11833
3,83253881,390782,131019,32139,49282,1,0.0,NaN,145,0.0,...,--- \n- - start\n - 1317738340042\n- - answer...,23057.0,tutor,ClassAssignment,NaN,NaN,NaN,NaN,NaN,5924
4,83255489,390782,131019,32120,49263,1,1.0,NaN,199,0.0,...,--- \n- - start\n - 1317738468101\n- - answer...,23038.0,tutor,ClassAssignment,NaN,NaN,NaN,NaN,NaN,5924


In [54]:
pl_df.columns

Index([u'id', u'assignment_id', u'user_id', u'assistment_id', u'problem_id',
       u'original', u'correct', u'answer_id', u'answer_text', u'first_action',
       u'hint_count', u'bottom_hint', u'attempt_count', u'start_time',
       u'end_time', u'first_response_time', u'overlap_time', u'actions',
       u'tutor_strategy_id', u'tutor_mode', u'assignment_type',
       u'teacher_comment', u'network_state', u'path_info', u'discrete_score',
       u'updated_at', u'sequence_id'],
      dtype='object')

In [56]:
len(pl_df)

390470

In [55]:
len(pl_df['sequence_id'].unique())

9157

In [82]:
counts = pd.value_counts(pl_df['sequence_id'])
mask = pl_df['sequence_id'].isin(counts[counts > 500].index)

In [80]:
new_df = pd.DataFrame()

In [83]:
pl_df['sequence_id'][~mask] = "-"
new_df = pd.get_dummies(pl_df['sequence_id'])

In [84]:
new_df.head()

,5582,5611,5613,5623,5625,5893,5897,5898,5918,5920,...,377658,377938,377942,381617,403378,459531,497963,581687,701680,-
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [85]:
# u'correct , u'bottom_hint, u'hint_count, u'attempt_count, u'first_response_time
new_df['correct'] = pl_df['correct']
new_df['bottom_hint'] = pl_df['bottom_hint']
new_df['hint_count'] = pl_df['hint_count']
new_df['attempt_count'] = pl_df['attempt_count']
new_df['first_response_time'] = pl_df['first_response_time']

In [109]:
new_df['user_id'] = pl_df['user_id']

In [114]:
new_df['id'] = pl_df['id']

In [88]:
new_df['bottom_hint'].fillna(0, inplace=True)

In [103]:
norm = lambda x: (x - x.min()) / (x.max() - x.min())

In [135]:
del new_df['first_response_time']

In [105]:
new_df.insert(len(new_df.columns), 'NofFRT', pl_df.groupby('sequence_id')['first_response_time'].transform(norm))

In [136]:
new_df.head()

,5582,5611,5613,5623,5625,5893,5897,5898,5918,5920,...,581687,701680,-,correct,bottom_hint,hint_count,attempt_count,NofFRT,user_id,id
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1.0,0.0,0.0,1,0.004730,131019,83251979
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1.0,0.0,0.0,1,0.004712,131019,83252617
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1.0,0.0,0.0,1,0.004712,131019,83253196
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0.0,0.0,0.0,2,0.004769,131019,83253881
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1.0,0.0,0.0,1,0.004753,131019,83255489


In [156]:
new_df['correct'].fillna(0, inplace=True)
new_df['NofFRT'].fillna(1, inplace=True)
new_df['hint_count'].fillna(1, inplace=True)

In [160]:
new_df.isnull().values.any()

False

In [112]:
len(new_df['user_id'].unique())

384

In [157]:
new_df.to_csv('../LSTM-autoencoder/train_data.csv', index=False)

In [138]:
train_df = pd.read_csv('../LSTM-autoencoder/train_data.csv')

In [130]:
train_df['id'].head()

0    83251979
1    83252617
2    83253196
3    83253881
4    83255489
Name: id, dtype: int64

In [141]:
id_df = train_df.set_index('id')
train_g = id_df.groupby('user_id')

In [151]:
train_g.get_group(131019).iloc[:1, :5].as_matrix()

array([[0, 0, 0, 0, 0]])

In [ ]:
    print group

In [125]:
len(train_df.columns)

187

In [69]:
len(counts[counts > 500])

178

In [39]:
pl_df.set_index('id', inplace=True)

In [40]:
pl_g = pl_df.groupby('user_id')

In [45]:
max_len = 0
cnt_list = []
for name,group in pl_g:
    cnt = len(group)
    if cnt > max_len:
        max_len = cnt
    cnt_list.append(cnt)
print max_len
print sum(cnt_list)/len(cnt_list)
print min(cnt_list)

5357
1213
19


In [47]:
np.median(np.array(cnt_list))

744.0

In [48]:
user_list = pl_df['user_id'].unique().tolist()

In [49]:
batch_size = 30
chunks = [user_list[x:x+batch_size] for x in xrange(0, len(user_list), batch_size)]

In [51]:
for c in chunks:
    l = []
    for ite in c:
        m = pl_g.get_group(ite).iloc[:, 1:].as_matrix()
        if max_len >= m.shape[0]:
            diff = max_len - m.shape[0]
            l.append(np.pad(m, ((0,diff), (0,0)), mode='constant', constant_values=0))
        else:
            l.append(m[-1*max_len:, :])
        #l.append(g.get_group(ite).iloc[:, 1:].as_matrix())
    
    batch = np.stack(l, axis=0)
    break

In [161]:
# play around with pandas
sample = pd.DataFrame(np.ones((3,3)), columns=['a', 'b', 'c'])

In [180]:
sample['a'] = sample['a'].apply(lambda x: x+1)

In [183]:
sample.iloc[1:, :].reset_index(drop=True)

,a,b,c
0,2.0,1.0,1.0
1,2.0,1.0,1.0


In [184]:
values = pd.Series(["a","b","a","b","c","d","e","a"])
counts = pd.value_counts(values)
mask = values.isin(counts[counts > 1].index)
print pd.get_dummies(values[mask])

   a  b
0  1  0
1  0  1
2  1  0
3  0  1
7  1  0


In [188]:
mask

0     True
1     True
2     True
3     True
4    False
5    False
6    False
7     True
dtype: bool